In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import sklearn.preprocessing
import sklearn.neural_network
import sklearn.model_selection

import glob

In [ ]:
datapoints = np.sort(glob.glob('Dataset/*datapoints.txt')) # Cada archivo contiene todos las instancias de una misma categoría
positive = np.sort(glob.glob('Dataset/*targets.txt')) # Indica si la instancia corresponde al gesto esperado o es sólo un gesto neutro
target = []
datos = []

for i in range(len(positive)):
    gesto = np.loadtxt(positive[i])
    categoria = np.loadtxt(datapoints[i],skiprows=1)[np.where(gesto==1)] #Coge sólo las instancias donde hay gesto
    categoria = categoria[:330] # Corta la cantidad de instancias, para que todas las categorías estén igualmente representadas
    for j in range(len(categoria)):
        dato = categoria[j]
        datos.append(dato[1:]) # Descarta el primer dato (no es relevante)
        target.append(i%8)
datos = np.array(datos,dtype=float)

In [ ]:
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(datos, target, test_size=0.2)

scaler = sklearn.preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
neuronas = np.linspace(1,20,5,dtype=int)
loss = []
f1_test = []
f1_train = []
it = 2500

for i in neuronas:
    mlp = sklearn.neural_network.MLPClassifier(activation='logistic',hidden_layer_sizes=(i),max_iter=it)
    mlp.fit(X_train, Y_train)
    
    loss.append(mlp.loss_)
    f1_test.append(sklearn.metrics.f1_score(Y_test, mlp.predict(X_test), average='macro'))
    f1_train.append(sklearn.metrics.f1_score(Y_train, mlp.predict(X_train), average='macro'))
    print(i)

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(15,5))
ax1.scatter(neuronas,loss,c='b',label='Loss')
ax2.scatter(neuronas,f1_test,c='r',label='Test')
ax2.scatter(neuronas,f1_train,c='b',label='Train')
ax1.set_title('Loss')
ax2.set_title('F1 Score')
ax1.set_xlabel('Número de neuronas')
ax2.set_xlabel('Número de neuronas')
ax1.set_xticks(neuronas[1::2])
ax2.set_xticks(neuronas[1::2])
ax2.legend()
#fig.savefig('loss_f1.png')

In [ ]:
best = 5
mlp = sklearn.neural_network.MLPClassifier(activation='logistic',hidden_layer_sizes=(best),max_iter=it)
mlp.fit(X_train, Y_train)


fig,axes = plt.subplots(1,best,figsize=(25,5))
for i in range(best):
    axes[i].imshow(mlp.coefs_[0][:,i].reshape(25,12)
    axes[i].set_title('Neurona {}'.format(i+1))

#plt.savefig('neuronas.png')